In [24]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import math

In [3]:
data=pd.read_csv('../Data/raw/train.csv',delimiter=',')

data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
#Dropping useless Columns
data=data[["target","text"]]

data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
#Dropping null values
print(data.isnull().sum())
data = data.dropna()
print(data.isnull().sum())

target    0
text      0
dtype: int64
target    0
text      0
dtype: int64


In [6]:
#removing links and special characters from the tweets
#replacing usernames with @user
def clean_text(text):
    text = re.sub(r'@\S+', '@user', text)
    text = re.sub(r'http\S+|www\S+', '', text)                           
    text = re.sub(r'#(\w+)', r'\1', text)                               
    text = re.sub(r'\s+', ' ', text).strip()                             
    return text

In [7]:
data['text'] = data['text'].apply(clean_text)

In [8]:
#after cleaning
data.head()

,target,text
0,0,@user - A that's a bummer. You shoulda got Dav...
1,0,is upset that he can't update his Facebook by ...
2,0,@user I dived many times for the ball. Managed...
3,0,my whole body feels itchy and like its on fire
4,0,"@user no, it's not behaving at all. i'm mad. w..."


In [15]:
#Saving the Cleaned Data
data.to_csv('../Data/processed/cleaned_train.csv', index=False)

In [19]:
negative_samples=data[data["target"]==0].reset_index(drop=True)
positive_samples=data[data["target"]==4].reset_index(drop=True)

In [22]:
negative_samples.head()
positive_samples.head()

,target,text
0,4,I LOVE @user u guys r the best!!
1,4,im meeting up with one of my besties tonight! ...
2,4,"@user Thanks for the Twitter add, Sunisa! I go..."
3,4,Being sick can be really cheap when it hurts t...
4,4,@user he has that effect on everyone


In [23]:
#Shuffling the samples
negative_samples = negative_samples.sample(frac=1, random_state=42)
positive_samples = positive_samples.sample(frac=1, random_state=42)

In [26]:
#making 1000 chunks of positive and negative samples each for annotation
num_per_class = 500
num_iterations = math.ceil(len(negative_samples)/num_per_class)

for i in range(num_iterations):
    neg_chunk = negative_samples[i*num_per_class:(i+1)*num_per_class]
    pos_chunk = positive_samples[i*num_per_class:(i+1)*num_per_class]
    chunk = pd.concat([neg_chunk, pos_chunk]).sample(frac=1, random_state=42).reset_index(drop=True)
    chunk.to_csv(f'../Data/processed/Chunks/chunk_{i+1}.csv', index=False)
